# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
weather_df = pd.read_csv("./Resources/weather_df.csv")
weather_df = weather_df[['City', 'Lat', 'Lng', 'Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]
weather_df.head()

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,23.67,42,96,5.10,US,1612807182
1,Erenhot,43.6475,111.9767,16.38,88,0,8.48,CN,1612807182
2,Kavaratti,10.5669,72.6420,82.08,62,6,7.49,IN,1612807182
3,Broken Hill,-31.9500,141.4333,66.20,26,0,8.05,AU,1612807183
4,Sisimiut,66.9395,-53.6735,22.05,80,0,12.33,GL,1612807183


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
gmaps.configure(api_key=g_key)
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]
fig = gmaps.figure(zoom_level=2, center=(10,0))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=weather_df['Humidity'].max(), point_radius=1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
ideal_df = weather_df.loc[(weather_df['Temp'] >= 63) & (weather_df['Temp'] <= 70)]
ideal_df = ideal_df.loc[(weather_df['Humidity'] >= 20) & (weather_df['Humidity'] <= 40)]
ideal_df = ideal_df.loc[weather_df['Wind Speed'] <= 15]
ideal_df.head()

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Broken Hill,-31.9500,141.4333,66.20,26,0,8.05,AU,1612807183
24,Punta Arenas,-53.1500,-70.9167,69.39,40,0,14.97,CL,1612807178
286,Yazman,29.1212,71.7446,63.77,25,3,4.92,PK,1612807295
340,Sangāreddi,17.6294,78.0917,68.00,37,0,4.09,IN,1612807311
367,Jalu,29.0331,21.5482,66.61,30,94,10.18,LY,1612807318


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""

hotels = []

for index, row in hotel_df.iterrows():
    
    lati = row["Lat"]
    long = row["Lng"]      
    
    params = {
    "location": f"{lati}, {long}",
    "keyword": "hotels",
    "radius": 5000,
    "type": 'lodging',
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    data = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "No Near Logdging Found"        

hotel_df.head()

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,Broken Hill,-31.9500,141.4333,66.20,26,0,8.05,AU,1612807183,Broken Hill Tourist Park
24,Punta Arenas,-53.1500,-70.9167,69.39,40,0,14.97,CL,1612807178,Hotel Dreams Del Estrecho
286,Yazman,29.1212,71.7446,63.77,25,3,4.92,PK,1612807295,No Near Logdging Found
340,Sangāreddi,17.6294,78.0917,68.00,37,0,4.09,IN,1612807311,OYO 71906 Grand Hotel And Lodge
367,Jalu,29.0331,21.5482,66.61,30,94,10.18,LY,1612807318,Jalu Hotel


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [53]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))